# Group 89
## Quinten van de Korput (student id: 105473), Anna Orlandi (student id: 105678)
 

In [16]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from scipy import stats
from sklearn.model_selection import GridSearchCV

### Model preprocessing

In [2]:
train_a = pd.read_parquet('data/A/train_targets.parquet')
train_b = pd.read_parquet('data/B/train_targets.parquet')
train_c = pd.read_parquet('data/C/train_targets.parquet')
X_train_estimated_a = pd.read_parquet('data/A/X_train_estimated.parquet')
X_train_estimated_b = pd.read_parquet('data/B/X_train_estimated.parquet')
X_train_estimated_c = pd.read_parquet('data/C/X_train_estimated.parquet')
X_train_observed_a = pd.read_parquet('data/A/X_train_observed.parquet')
X_train_observed_b = pd.read_parquet('data/B/X_train_observed.parquet')
X_train_observed_c = pd.read_parquet('data/C/X_train_observed.parquet')
X_test_estimated_a = pd.read_parquet('data/A/X_test_estimated.parquet')
X_test_estimated_b = pd.read_parquet('data/B/X_test_estimated.parquet')
X_test_estimated_c = pd.read_parquet('data/C/X_test_estimated.parquet')

In [3]:
train_b.loc[(train_b['time'] >= "2019-05-31 07:00:00") & (train_b['time'] <= "2019-06-03 12:00:00"), 'pv_measurement'] = np.NaN
train_b.loc[("2019-03-24 00:00:00" <= train_b["time"]) & (train_b["time"] <= "2019-03-27 00:00:00"), 'pv_measurement'] = np.NaN
train_b.loc[("2019-10-28 13:00:00" <= train_b["time"]) & (train_b["time"] <= "2019-10-30 22:00:00"), 'pv_measurement'] = np.NaN
train_b.loc[("2020-03-26 13:00:00" <= train_b["time"]) & (train_b["time"] <= "2020-03-27 21:00:00"), 'pv_measurement'] = np.NaN
train_b.loc[(train_b['time'] >= "2020-04-02 01:00:00") & (train_b['time'] <= "2020-04-16 06:00:00"), 'pv_measurement'] = np.NaN
train_b.loc[(train_b['time'] >= "2020-07-12 20:00:00") & (train_b['time'] <= "2020-08-25 21:00:00"), 'pv_measurement'] = np.NaN
train_b.loc[("2020-09-24 12:00:00" <= train_b["time"]) & (train_b["time"] <= "2020-09-25 21:00:00"), 'pv_measurement'] = np.NaN
train_b.loc[(train_b['time'] >= "2021-02-17 23:00:00") & (train_b['time'] <= "2021-04-19 11:00:00"), 'pv_measurement'] = np.NaN
train_b.loc[("2021-04-28 22:00:00" <= train_b["time"]) & (train_b["time"] <= "2021-05-01 21:00:00"), 'pv_measurement'] = np.NaN
train_b.loc[("2021-06-05 01:00:00" <= train_b["time"]) & (train_b["time"] <= "2021-06-07 07:00:00"), 'pv_measurement'] = np.NaN
train_b.loc[("2021-06-13 01:00:00" <= train_b["time"]) & (train_b["time"] <= "2021-06-14 09:00:00"), 'pv_measurement'] = np.NaN
train_b.loc[("2021-06-22 01:00:00" <= train_b["time"]) & (train_b["time"] <= "2021-06-24 08:00:00"), 'pv_measurement'] = np.NaN
train_b.loc[("2021-07-03 12:00:00" <= train_b["time"]) & (train_b["time"] <= "2021-07-06 05:00:00"), 'pv_measurement'] = np.NaN
train_b.loc[("2021-08-25 22:00:00" <= train_b["time"]) & (train_b["time"] <= "2021-09-03 21:00:00"), 'pv_measurement'] = np.NaN
train_b.loc[("2021-09-08 12:00:00" <= train_b["time"]) & (train_b["time"] <= "2021-09-14 13:00:00"), 'pv_measurement'] = np.NaN
train_b.loc[("2021-09-18 23:00:00" <= train_b["time"]) & (train_b["time"] <= "2021-09-27 08:00:00"), 'pv_measurement'] = np.NaN
train_b.loc[(train_b['time'] >= "2022-03-19 13:00:00") & (train_b['time'] <= "2022-04-14 06:00:00"), 'pv_measurement'] = np.NaN

In [4]:
X_train_observed_a["merge"] = X_train_observed_a["date_forecast"].dt.floor("60min")
X_train_observed_b["merge"] = X_train_observed_b["date_forecast"].dt.floor("60min")
X_train_observed_c["merge"] = X_train_observed_c["date_forecast"].dt.floor("60min")
X_train_estimated_a["merge"] = X_train_estimated_a["date_forecast"].dt.floor("60min")
X_train_estimated_b["merge"] = X_train_estimated_b["date_forecast"].dt.floor("60min")
X_train_estimated_c["merge"] = X_train_estimated_c["date_forecast"].dt.floor("60min")
train_a["merge"] = train_a["time"].dt.floor("60min")
train_b["merge"] = train_b["time"].dt.floor("60min")
train_c["merge"] = train_c["time"].dt.floor("60min")

In [5]:
def merge_target_train(df_target, df_train_observed, df_train_estimated):
    df_target["pv_measurement"] = df_target["pv_measurement"] / 4
    df = pd.concat([df_train_observed, df_train_estimated], axis=0)
    df = pd.merge(left=df_target, right=df, on="merge")
    df.drop(["merge"], axis=1, inplace=True)
    # df = pd.merge(left=df_target, right=df, left_on="time", right_on="date_forecast")
    return df

In [6]:
Train_a = merge_target_train(train_a, X_train_observed_a, X_train_estimated_a)
Train_b = merge_target_train(train_b, X_train_observed_b, X_train_estimated_b)
Train_c = merge_target_train(train_c, X_train_observed_c, X_train_estimated_c)

In [7]:
changes_list = Train_a.columns.drop(["date_calc", "time", "date_forecast", 'precip_type_5min:idx', 'snow_drift:idx', 'is_in_shadow:idx', 
                                                 'is_day:idx', 'dew_or_rime:idx', 'ceiling_height_agl:m', 'snow_density:kgm3', 
                                                 'elevation:m', 'cloud_base_agl:m'])
Train_a = Train_a[~((np.abs(stats.zscore(Train_a[changes_list], axis=1))) < 3).all(axis=1)]
Train_b = Train_b[~((np.abs(stats.zscore(Train_b[changes_list], axis=1))) < 3).all(axis=1)]
Train_c = Train_c[~((np.abs(stats.zscore(Train_c[changes_list], axis=1))) < 3).all(axis=1)]

In [8]:
def feature_engineering(df_in):
    df = df_in.copy()
    # Fill date calc with date forecast when null
    df["date_calc"].fillna(df["date_forecast"], inplace=True)
    
    # Make time data
    df["hour"] = df["date_forecast"].dt.hour
    df["month"] = df["date_forecast"].dt.month
    df["minutes"] = df["date_forecast"].dt.minute
    df["day"] = df["date_forecast"].dt.day
    df["dayofyear"] = df["date_forecast"].dt.dayofyear
    df["hours_since_calc"] = (df["date_forecast"] - df["date_calc"]).dt.total_seconds() / (60*60)

    df["SumRad"] = df["direct_rad:W"] + df["diffuse_rad:W"]
    df["ClearSkyRatio"] = df["SumRad"]/(1+df["clear_sky_rad:W"])
    df["SunElevationRatio"] = df["sun_elevation:d"]/df["elevation:m"]
    df["TempDewPoint"] = df["t_1000hPa:K"] - df["dew_point_2m:K"]
    # df["SnowFall"] = df["fresh_snow_1h:cm"] + df["fresh_snow_3h:cm"] + df["fresh_snow_6h:cm"] + df["fresh_snow_12h:cm"]
    
    # df["Date"] = df["date_forecast"].dt.date
    # df["SunAboveHorizon"] = df["sun_elevation:d"] > 0
    # Selection = df[["Date", "location", "SunAboveHorizon"]].groupby(["Date", "location"]).sum()
    # df.drop("SunAboveHorizon", inplace=True, axis=1)
    # df = pd.merge(left=df, right=Selection, on=["Date", "location"])
    
    # Drop time data
    df.drop(["date_forecast", "date_calc"], axis=1, inplace=True)
    
    # One hot encoding categorical data
    df = pd.get_dummies(df)
    return df

In [9]:
Train_a["location"] = "A"
Train_b["location"] = "B"
Train_c["location"] = "C"
Train_b = Train_b.loc[~Train_b["pv_measurement"].isnull()]
Train_c = Train_c.loc[~Train_c["pv_measurement"].isnull()]

In [10]:
def feature_selection(df_in):
    df = df_in.copy()

    df.drop(["prob_rime:p", "dew_or_rime:idx", "fresh_snow_6h:cm"], axis=1, inplace=True)
    return df

In [11]:
A = feature_selection(feature_engineering(Train_a.sort_values(by="time")))
B = feature_selection(feature_engineering(Train_b.sort_values(by="time")))
C = feature_selection(feature_engineering(Train_c.sort_values(by="time")))

In [12]:
A.loc[A["is_day:idx"] == 0, "pv_measurement"] = 0
B.loc[B["is_day:idx"] == 0, "pv_measurement"] = 0
C.loc[C["is_day:idx"] == 0, "pv_measurement"] = 0

### Model A

In [59]:
train_selection_a = A["time"] < "2023-03-01"
Train_A.shape, Test_A.shape

((129665, 55), (5855, 55))

In [60]:
X_a = A.drop(["time", "pv_measurement"], axis=1)
y_a = A["pv_measurement"]

In [61]:
# Model_data_features.reset_index(inplace=True)
train_selection_2 = A["time"] < "2023-01-01"
test_selection_2 = (A["time"] > "2023-01-01") & (A["time"] < "2023-03-01")
train_selection_3 = A["time"] < "2022-11-01"
test_selection_3 = (A["time"] > "2022-11-01") & (A["time"] < "2023-01-01")
Train_1_indices = A.reset_index().index[train_selection_a]
Test_1_indices = A.reset_index().index[~train_selection_a]
Train_2_indices = A.reset_index().index[train_selection_2]
Test_2_indices = A.reset_index().index[test_selection_2]
Train_3_indices = A.reset_index().index[train_selection_3]
Test_3_indices = A.reset_index().index[test_selection_3]

In [62]:
param_grid = {
    'n_estimators': [119],
    'learning_rate': [0.06],
    'max_depth': [8]
}

In [63]:
gridsearch = GridSearchCV(xgb.XGBRegressor(n_estimators=120, learning_rate=0.065, max_depth=11, min_child_weight=1, gamma=0.3, reg_alpha=0.01, reg_lambda=1,
                       enable_categorical=True, eval_metric="mae", random_state=0, colsample_bytree=0.8, subsample=0.8), 
                          param_grid, scoring="neg_mean_absolute_error", 
                          verbose=2, n_jobs=-1, return_train_score=True, cv=[(Train_1_indices, Test_1_indices), (Train_2_indices, Test_2_indices), (Train_3_indices, Test_3_indices)])

In [64]:
import time
start = time.time()
gridsearch.fit(X_a, y_a)
print("time:", time.time() - start)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
time: 6.389873266220093


In [65]:
gridsearch.best_params_

{'learning_rate': 0.06, 'max_depth': 8, 'n_estimators': 119}

In [66]:
gridsearch.cv_results_

{'mean_fit_time': array([3.95945986]),
 'std_fit_time': array([0.09282453]),
 'mean_score_time': array([0.02246936]),
 'std_score_time': array([0.00043581]),
 'param_learning_rate': masked_array(data=[0.06],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_max_depth': masked_array(data=[8],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[119],
              mask=[False],
        fill_value='?',
             dtype=object),
 'params': [{'learning_rate': 0.06, 'max_depth': 8, 'n_estimators': 119}],
 'split0_test_score': array([-60.39180114]),
 'split1_test_score': array([-13.49927781]),
 'split2_test_score': array([-14.04506741]),
 'mean_test_score': array([-29.31204879]),
 'std_test_score': array([21.97783317]),
 'rank_test_score': array([1], dtype=int32),
 'split0_train_score': array([-30.72504791]),
 'split1_train_score': array([-32.02861475]),
 'split2_train_score': array([

In [67]:
gridsearch.best_score_

-29.312048788489573

In [68]:
reg_a = xgb.XGBRegressor(n_estimators=119, learning_rate=0.06, max_depth=8, min_child_weight=1, gamma=0.3, reg_alpha=0.01, reg_lambda=1,
                       enable_categorical=True, eval_metric="mae", random_state=0, colsample_bytree=0.8, subsample=0.8)
reg_a.fit(X_a, y_a, verbose=True)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=True, eval_metric='mae', feature_types=None,
             gamma=0.3, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.06, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=119, n_jobs=None,
             num_parallel_tree=None, random_state=0, ...)

### Model B

In [69]:
train_selection_b = B["time"] < "2023-03-01"
Train_b.shape, Test_b.shape

((106931, 55), (5415, 55))

In [70]:
X_b = B.drop(["time", "pv_measurement"], axis=1)
y_b = B["pv_measurement"]

In [71]:
# Model_data_features.reset_index(inplace=True)
train_selection_2 = B["time"] < "2023-01-01"
test_selection_2 = (B["time"] > "2023-01-01") & (B["time"] < "2023-03-01")
train_selection_3 = B["time"] < "2022-11-01"
test_selection_3 = (B["time"] > "2022-11-01") & (B["time"] < "2023-01-01")
Train_1_indices = B.reset_index().index[train_selection_b]
Test_1_indices = B.reset_index().index[~train_selection_b]
Train_2_indices = B.reset_index().index[train_selection_2]
Test_2_indices = B.reset_index().index[test_selection_2]
Train_3_indices = B.reset_index().index[train_selection_3]
Test_3_indices = B.reset_index().index[test_selection_3]

In [72]:
param_grid = {
    'n_estimators': [114],
    'learning_rate': [0.06],
    'max_depth': [9]
}

In [73]:
gridsearch = GridSearchCV(xgb.XGBRegressor(n_estimators=120, learning_rate=0.065, max_depth=11, min_child_weight=1, gamma=0.3, reg_alpha=0.01, reg_lambda=1,
                       enable_categorical=True, eval_metric="mae", random_state=0, colsample_bytree=0.8, subsample=0.8), 
                          param_grid, scoring="neg_mean_absolute_error", 
                          verbose=2, n_jobs=-1, return_train_score=True, cv=[(Train_1_indices, Test_1_indices), (Train_2_indices, Test_2_indices), (Train_3_indices, Test_3_indices)])

In [74]:
import time
start = time.time()
gridsearch.fit(X_b, y_b)
print("time:", time.time() - start)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
time: 7.914872884750366


In [75]:
gridsearch.best_params_

{'learning_rate': 0.06, 'max_depth': 9, 'n_estimators': 114}

In [76]:
gridsearch.cv_results_

{'mean_fit_time': array([4.8152771]),
 'std_fit_time': array([0.07442341]),
 'mean_score_time': array([0.02446914]),
 'std_score_time': array([0.00298373]),
 'param_learning_rate': masked_array(data=[0.06],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_max_depth': masked_array(data=[9],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[114],
              mask=[False],
        fill_value='?',
             dtype=object),
 'params': [{'learning_rate': 0.06, 'max_depth': 9, 'n_estimators': 114}],
 'split0_test_score': array([-9.08800173]),
 'split1_test_score': array([-1.31531187]),
 'split2_test_score': array([-0.63844329]),
 'mean_test_score': array([-3.68058563]),
 'std_test_score': array([3.83359269]),
 'rank_test_score': array([1], dtype=int32),
 'split0_train_score': array([-2.78081315]),
 'split1_train_score': array([-2.90493827]),
 'split2_train_score': array([-2.92377

In [77]:
gridsearch.best_score_

-3.680585629956051

In [78]:
reg_b = xgb.XGBRegressor(n_estimators=119, learning_rate=0.06, max_depth=8, min_child_weight=1, gamma=0.3, reg_alpha=0.01, reg_lambda=1,
                       enable_categorical=True, eval_metric="mae", random_state=0, colsample_bytree=0.8, subsample=0.8)
reg_b.fit(X_b, y_b, verbose=True)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=True, eval_metric='mae', feature_types=None,
             gamma=0.3, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.06, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=119, n_jobs=None,
             num_parallel_tree=None, random_state=0, ...)

### Model C

In [79]:
train_selection_c = C["time"] < "2023-03-01"
Train_c.shape, Test_c.shape

((98975, 55), (4612, 55))

In [81]:
X_c = C.drop(["time", "pv_measurement"], axis=1)
y_c = C["pv_measurement"]

In [82]:
# Model_data_features.reset_index(inplace=True)
train_selection_2 = C["time"] < "2023-01-01"
test_selection_2 = (C["time"] > "2023-01-01") & (C["time"] < "2023-03-01")
train_selection_3 = C["time"] < "2022-11-01"
test_selection_3 = (C["time"] > "2022-11-01") & (C["time"] < "2023-01-01")
Train_1_indices = C.reset_index().index[train_selection_c]
Test_1_indices = C.reset_index().index[~train_selection_c]
Train_2_indices = C.reset_index().index[train_selection_2]
Test_2_indices = C.reset_index().index[test_selection_2]
Train_3_indices = C.reset_index().index[train_selection_3]
Test_3_indices = C.reset_index().index[test_selection_3]

In [83]:
param_grid = {
    'n_estimators': [114],
    'learning_rate': [0.06],
    'max_depth': [9]
}

In [84]:
gridsearch = GridSearchCV(xgb.XGBRegressor(n_estimators=120, learning_rate=0.065, max_depth=11, min_child_weight=1, gamma=0.3, reg_alpha=0.01, reg_lambda=1,
                       enable_categorical=True, eval_metric="mae", random_state=0, colsample_bytree=0.8, subsample=0.8), 
                          param_grid, scoring="neg_mean_absolute_error", 
                          verbose=2, n_jobs=-1, return_train_score=True, cv=[(Train_1_indices, Test_1_indices), (Train_2_indices, Test_2_indices), (Train_3_indices, Test_3_indices)])

In [85]:
import time
start = time.time()
gridsearch.fit(X_c, y_c)
print("time:", time.time() - start)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
time: 7.185922384262085


In [86]:
gridsearch.best_params_

{'learning_rate': 0.06, 'max_depth': 9, 'n_estimators': 114}

In [87]:
gridsearch.cv_results_

{'mean_fit_time': array([4.52992535]),
 'std_fit_time': array([0.06941905]),
 'mean_score_time': array([0.01849961]),
 'std_score_time': array([0.00194879]),
 'param_learning_rate': masked_array(data=[0.06],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_max_depth': masked_array(data=[9],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[114],
              mask=[False],
        fill_value='?',
             dtype=object),
 'params': [{'learning_rate': 0.06, 'max_depth': 9, 'n_estimators': 114}],
 'split0_test_score': array([-11.47863507]),
 'split1_test_score': array([-1.56188937]),
 'split2_test_score': array([-0.8774979]),
 'mean_test_score': array([-4.63934078]),
 'std_test_score': array([4.84417572]),
 'rank_test_score': array([1], dtype=int32),
 'split0_train_score': array([-2.38738764]),
 'split1_train_score': array([-2.46797993]),
 'split2_train_score': array([-2.5175

In [88]:
gridsearch.best_score_

-4.639340780799683

In [89]:
reg_c = xgb.XGBRegressor(n_estimators=112, learning_rate=0.06, max_depth=7, min_child_weight=1, gamma=0.3, reg_alpha=0.01, reg_lambda=1,
                       enable_categorical=True, eval_metric="mae", random_state=0, colsample_bytree=0.8, subsample=0.8)
reg_c.fit(X_c, y_c, verbose=True)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=True, eval_metric='mae', feature_types=None,
             gamma=0.3, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.06, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=7, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=112, n_jobs=None,
             num_parallel_tree=None, random_state=0, ...)

### Prediction

In [90]:
X_test_estimated_a["location"] = "A"
X_test_estimated_b["location"] = "B"
X_test_estimated_c["location"] = "C"

In [91]:
Test_X_a = feature_selection(feature_engineering(X_test_estimated_a))
Test_X_b = feature_selection(feature_engineering(X_test_estimated_b))
Test_X_c = feature_selection(feature_engineering(X_test_estimated_c))

In [92]:
y_pred_a = reg_a.predict(Test_X_a)
y_pred_b = reg_b.predict(Test_X_b)
y_pred_c = reg_c.predict(Test_X_c)

In [93]:
y_pred_a[(Test_X_a["is_day:idx"] == 0)] = 0
y_pred_b[(Test_X_b["is_day:idx"] == 0)] = 0
y_pred_c[(Test_X_c["is_day:idx"] == 0)] = 0

In [94]:
y_pred = np.concatenate([y_pred_a, y_pred_b, y_pred_c])

In [95]:
y_pred = (y_pred[::4] + y_pred[1::4] + y_pred[2::4] + y_pred[3::4])
y_pred[:20]

array([   0.      ,    0.      ,    0.      ,   53.68991 ,  364.0292  ,
        991.60925 , 2008.198   , 2531.7986  , 2587.5217  , 2710.782   ,
       2940.7344  , 3040.8008  , 2757.8179  , 2533.2622  , 2364.9495  ,
       1669.5106  , 1080.4099  ,  622.8436  ,  221.38722 ,   15.455173],
      dtype=float32)

In [56]:
threshold = 0.05

In [98]:
result_df = pd.DataFrame(y_pred, columns=["prediction"])
result_df["prediction"] = result_df["prediction"].apply(lambda x:0 if x < threshold else x)
print(result_df["prediction"])
result_df.to_csv("short_notebook_1.csv", index=True, index_label="id")

0         0.000000
1         0.000000
2         0.000000
3        53.689911
4       364.029205
           ...    
2155     82.937698
2156     62.373196
2157      7.236816
2158      0.000000
2159      0.000000
Name: prediction, Length: 2160, dtype: float64
[CV] END ..learning_rate=0.06, max_depth=9, n_estimators=114; total time=   4.5s
[CV] END ..learning_rate=0.06, max_depth=9, n_estimators=114; total time=   4.5s
[CV] END ..learning_rate=0.06, max_depth=9, n_estimators=114; total time=   4.6s
[CV] END ..learning_rate=0.06, max_depth=8, n_estimators=119; total time=   3.9s
[CV] END ..learning_rate=0.06, max_depth=8, n_estimators=119; total time=   4.0s
[CV] END ..learning_rate=0.06, max_depth=8, n_estimators=119; total time=   4.1s
[CV] END ..learning_rate=0.06, max_depth=8, n_estimators=119; total time=   4.8s
[CV] END ..learning_rate=0.06, max_depth=9, n_estimators=114; total time=   4.8s
[CV] END ..learning_rate=0.06, max_depth=8, n_estimators=119; total time=   4.6s
[CV] END ..lear